# Aggregation Features Analysis for Behavioral Biometrics Detection

This notebook focuses on extracting and analyzing aggregation features from the static analysis data to improve machine learning model performance for detecting behavioral biometrics scripts.

## Key Objectives:
1. Extract aggregation features from the database
2. Create vendor-agnostic aggregation features
3. Analyze feature importance and vendor correlation
4. Integrate best aggregation features with existing model
5. Compare performance with and without aggregation features

In [1]:
# Cell 1: Imports and Database Connection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
import psycopg2
import json
import warnings
from collections import Counter, defaultdict
warnings.filterwarnings('ignore')

# Database connection
def load_aggregation_data():
    conn = psycopg2.connect(
        host="localhost",
        port=5434,
        database="vv8_backend",
        user="vv8",
        password="vv8"
    )
    
    # Load all aggregation-related columns
    query = """
    SELECT 
        script_id,
        max_api_aggregation_score,
        behavioral_api_agg_count,
        fp_api_agg_count,
        max_aggregated_apis,
        max_behavioral_api_aggregation_score,
        aggregated_behavioral_apis,
        max_fingerprinting_api_aggregation_score,
        aggregated_fingerprinting_apis,
        attached_listeners,
        dataflow_to_sink,
        apis_going_to_sink,
        graph_construction_failure,
        label,
        vendor,
        -- Include original features for comparison
        fingerprinting_source_apis,
        behavioral_source_apis,
        behavioral_apis_access_count,
        fingerprinting_api_access_count
    FROM multicore_static_info_known_companies
    """
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    print(f"Loaded {len(df)} scripts with aggregation features")
    return df

# Load the data
df = load_aggregation_data()
print(f"\nDataset shape: {df.shape}")
print(f"Label distribution: {df['label'].value_counts().to_dict()}")

Loaded 2229 scripts with aggregation features

Dataset shape: (2229, 19)
Label distribution: {-1: 1753, 0: 244, 1: 232}


In [2]:
# Cell 2: Explore Aggregation Features
print("=== AGGREGATION FEATURES OVERVIEW ===")

# Basic statistics
agg_columns = [
    'max_api_aggregation_score', 'behavioral_api_agg_count', 'fp_api_agg_count',
    'max_behavioral_api_aggregation_score', 'max_fingerprinting_api_aggregation_score'
]

print("\n📊 Numerical Aggregation Features Statistics:")
print(df[agg_columns].describe())

# Analyze by label
print("\n📈 Aggregation Features by Label:")
for label in sorted(df['label'].unique()):
    subset = df[df['label'] == label]
    print(f"\n--- Label {label} ({len(subset)} samples) ---")
    print(subset[agg_columns].describe().round(2))

# Check for missing values
print("\n🔍 Missing Values Analysis:")
missing_analysis = df.isnull().sum()
print(missing_analysis[missing_analysis > 0])

# Analyze JSON columns
json_columns = ['max_aggregated_apis', 'aggregated_behavioral_apis', 'aggregated_fingerprinting_apis', 
                'attached_listeners', 'apis_going_to_sink']

print("\n📋 JSON Columns Analysis:")
for col in json_columns:
    non_null = df[col].notna().sum()
    pct = (non_null / len(df)) * 100
    print(f"{col}: {non_null:,} non-null ({pct:.1f}%)")

=== AGGREGATION FEATURES OVERVIEW ===

📊 Numerical Aggregation Features Statistics:
       max_api_aggregation_score  behavioral_api_agg_count  fp_api_agg_count  \
count                2229.000000               2229.000000       2229.000000   
mean                    5.484522                  3.091521          2.202782   
std                     9.514723                  7.821486          5.951794   
min                    -1.000000                 -1.000000         -1.000000   
25%                     1.000000                  0.000000          0.000000   
50%                     2.000000                  1.000000          0.000000   
75%                     6.000000                  3.000000          2.000000   
max                    82.000000                 82.000000         66.000000   

       max_behavioral_api_aggregation_score  \
count                           2229.000000   
mean                               3.742934   
std                                7.814780   
min    

In [3]:
# Cell 3: Create Vendor-Agnostic Aggregation Features
def create_aggregation_features(df):
    """
    Create vendor-agnostic features from aggregation data
    """
    features_list = []
    
    for idx, row in df.iterrows():
        try:
            features = {}
            
            # === BASIC AGGREGATION METRICS ===
            
            # Raw aggregation scores (handle -1 as missing/no aggregation)
            max_agg = row['max_api_aggregation_score'] if row['max_api_aggregation_score'] != -1 else 0
            behavioral_agg = row['behavioral_api_agg_count'] if row['behavioral_api_agg_count'] != -1 else 0
            fp_agg = row['fp_api_agg_count'] if row['fp_api_agg_count'] != -1 else 0
            
            features['max_api_aggregation_score'] = max_agg
            features['behavioral_api_agg_count'] = behavioral_agg
            features['fp_api_agg_count'] = fp_agg
            
            # Derived aggregation metrics
            total_agg = behavioral_agg + fp_agg
            features['total_aggregation_count'] = total_agg
            
            # Aggregation ratios (avoid division by zero)
            if total_agg > 0:
                features['behavioral_agg_ratio'] = behavioral_agg / total_agg
                features['fp_agg_ratio'] = fp_agg / total_agg
            else:
                features['behavioral_agg_ratio'] = 0
                features['fp_agg_ratio'] = 0
            
            # Aggregation intensity categories
            features['has_aggregation'] = int(max_agg > 0)
            features['has_behavioral_aggregation'] = int(behavioral_agg > 0)
            features['has_fp_aggregation'] = int(fp_agg > 0)
            features['has_both_aggregation_types'] = int(behavioral_agg > 0 and fp_agg > 0)
            
            # Aggregation complexity tiers
            if max_agg == 0:
                features['aggregation_complexity_tier'] = 0
            elif max_agg <= 2:
                features['aggregation_complexity_tier'] = 1
            elif max_agg <= 10:
                features['aggregation_complexity_tier'] = 2
            elif max_agg <= 25:
                features['aggregation_complexity_tier'] = 3
            else:
                features['aggregation_complexity_tier'] = 4
            
            # === AGGREGATED APIS ANALYSIS ===
            
            # Parse aggregated APIs
            max_apis = row['max_aggregated_apis'] if pd.notna(row['max_aggregated_apis']) else []
            behavioral_apis = row['aggregated_behavioral_apis'] if pd.notna(row['aggregated_behavioral_apis']) else []
            fp_apis = row['aggregated_fingerprinting_apis'] if pd.notna(row['aggregated_fingerprinting_apis']) else []
            
            # Convert to lists if they're strings
            if isinstance(max_apis, str):
                try:
                    max_apis = json.loads(max_apis)
                except:
                    max_apis = []
            if isinstance(behavioral_apis, str):
                try:
                    behavioral_apis = json.loads(behavioral_apis)
                except:
                    behavioral_apis = []
            if isinstance(fp_apis, str):
                try:
                    fp_apis = json.loads(fp_apis)
                except:
                    fp_apis = []
            
            # API diversity metrics
            features['unique_aggregated_apis_count'] = len(max_apis) if max_apis else 0
            features['unique_behavioral_agg_apis_count'] = len(behavioral_apis) if behavioral_apis else 0
            features['unique_fp_agg_apis_count'] = len(fp_apis) if fp_apis else 0
            
            # API category analysis (vendor-agnostic patterns)
            if max_apis:
                api_strings = [str(api) for api in max_apis]
                
                # Event-based aggregation patterns
                features['aggregates_mouse_events'] = int(any('Mouse' in api for api in api_strings))
                features['aggregates_keyboard_events'] = int(any('Keyboard' in api for api in api_strings))
                features['aggregates_touch_events'] = int(any('Touch' in api for api in api_strings))
                features['aggregates_pointer_events'] = int(any('Pointer' in api for api in api_strings))
                features['aggregates_focus_events'] = int(any('Focus' in api for api in api_strings))
                
                # Property aggregation patterns
                features['aggregates_coordinates'] = int(any(coord in api for api in api_strings 
                                                           for coord in ['clientX', 'clientY', 'screenX', 'screenY', 'pageX', 'pageY']))
                features['aggregates_timing'] = int(any('timeStamp' in api or 'time' in api.lower() for api in api_strings))
                features['aggregates_device_info'] = int(any(device in api for api in api_strings 
                                                           for device in ['Device', 'Hardware', 'Battery']))
                
                # Browser/environment aggregation
                features['aggregates_navigator'] = int(any('Navigator' in api for api in api_strings))
                features['aggregates_screen'] = int(any('Screen' in api for api in api_strings))
                features['aggregates_window'] = int(any('Window' in api for api in api_strings))
                features['aggregates_canvas'] = int(any('Canvas' in api or 'WebGL' in api for api in api_strings))
                
                # Aggregation diversity score
                event_types = sum([
                    features['aggregates_mouse_events'],
                    features['aggregates_keyboard_events'],
                    features['aggregates_touch_events'],
                    features['aggregates_pointer_events'],
                    features['aggregates_focus_events']
                ])
                
                property_types = sum([
                    features['aggregates_coordinates'],
                    features['aggregates_timing'],
                    features['aggregates_device_info']
                ])
                
                browser_types = sum([
                    features['aggregates_navigator'],
                    features['aggregates_screen'],
                    features['aggregates_window'],
                    features['aggregates_canvas']
                ])
                
                features['aggregation_event_diversity'] = event_types
                features['aggregation_property_diversity'] = property_types
                features['aggregation_browser_diversity'] = browser_types
                features['total_aggregation_diversity'] = event_types + property_types + browser_types
                
            else:
                # No aggregated APIs
                for feature_name in [
                    'aggregates_mouse_events', 'aggregates_keyboard_events', 'aggregates_touch_events',
                    'aggregates_pointer_events', 'aggregates_focus_events', 'aggregates_coordinates',
                    'aggregates_timing', 'aggregates_device_info', 'aggregates_navigator',
                    'aggregates_screen', 'aggregates_window', 'aggregates_canvas'
                ]:
                    features[feature_name] = 0
                
                features['aggregation_event_diversity'] = 0
                features['aggregation_property_diversity'] = 0
                features['aggregation_browser_diversity'] = 0
                features['total_aggregation_diversity'] = 0
            
            # === DATA FLOW AND SINK FEATURES ===
            
            features['has_dataflow_to_sink'] = int(row['dataflow_to_sink'] == True)
            features['has_graph_construction_failure'] = int(row['graph_construction_failure'] == True)
            
            # APIs going to sink
            sink_apis = row['apis_going_to_sink'] if pd.notna(row['apis_going_to_sink']) else {}
            if isinstance(sink_apis, str):
                try:
                    sink_apis = json.loads(sink_apis)
                except:
                    sink_apis = {}
            
            features['sink_apis_count'] = len(sink_apis) if sink_apis else 0
            features['has_sink_apis'] = int(len(sink_apis) > 0 if sink_apis else False)
            
            # Attached listeners
            listeners = row['attached_listeners'] if pd.notna(row['attached_listeners']) else []
            if isinstance(listeners, str):
                try:
                    listeners = json.loads(listeners)
                except:
                    listeners = []
            
            features['attached_listeners_count'] = len(listeners) if listeners else 0
            features['has_attached_listeners'] = int(len(listeners) > 0 if listeners else False)
            
            # === AGGREGATION vs NON-AGGREGATION COMPARISON ===
            
            # Compare with original source API counts
            behavioral_sources = row['behavioral_source_apis'] if pd.notna(row['behavioral_source_apis']) else []
            fp_sources = row['fingerprinting_source_apis'] if pd.notna(row['fingerprinting_source_apis']) else []
            
            if isinstance(behavioral_sources, str):
                try:
                    behavioral_sources = json.loads(behavioral_sources)
                except:
                    behavioral_sources = []
            if isinstance(fp_sources, str):
                try:
                    fp_sources = json.loads(fp_sources)
                except:
                    fp_sources = []
            
            behavioral_source_count = len(behavioral_sources) if behavioral_sources else 0
            fp_source_count = len(fp_sources) if fp_sources else 0
            
            # Aggregation efficiency ratios
            if behavioral_source_count > 0:
                features['behavioral_aggregation_efficiency'] = behavioral_agg / behavioral_source_count
            else:
                features['behavioral_aggregation_efficiency'] = 0
                
            if fp_source_count > 0:
                features['fp_aggregation_efficiency'] = fp_agg / fp_source_count
            else:
                features['fp_aggregation_efficiency'] = 0
            
            # Overall aggregation efficiency
            total_sources = behavioral_source_count + fp_source_count
            if total_sources > 0:
                features['overall_aggregation_efficiency'] = total_agg / total_sources
            else:
                features['overall_aggregation_efficiency'] = 0
            
            # Store metadata
            features['script_id'] = int(row['script_id'])
            features['label'] = int(row['label'])
            features['vendor'] = row['vendor'] if pd.notna(row['vendor']) else 'negative'
            
            features_list.append(features)
            
        except Exception as e:
            print(f"Error processing script {row.get('script_id', 'unknown')}: {e}")
            continue
    
    return pd.DataFrame(features_list)

# Create aggregation features
print("Creating vendor-agnostic aggregation features...")
agg_features_df = create_aggregation_features(df)

# Get feature columns (exclude metadata)
agg_feature_cols = [col for col in agg_features_df.columns if col not in ['script_id', 'label', 'vendor']]
print(f"Created {len(agg_feature_cols)} aggregation features for {len(agg_features_df)} scripts")

# Show sample of features
print(f"\nSample aggregation features:")
print(agg_features_df[agg_feature_cols[:10]].head())

Creating vendor-agnostic aggregation features...
Error processing script 7392023: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392036: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392015: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392039: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392076: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392075: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392171: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error processing script 7392005: The truth value of an array with more than 

In [4]:
# Cell 4: Analyze Aggregation Feature Distributions
print("=== AGGREGATION FEATURES ANALYSIS ===")

# Filter to binary classification (remove label -1)
binary_agg_df = agg_features_df[agg_features_df['label'].isin([0, 1])].copy()
print(f"\nFiltered to binary classification: {len(binary_agg_df)} samples")
print(f"Label distribution: {binary_agg_df['label'].value_counts().to_dict()}")

# Compare aggregation features between positive and negative samples
positive_samples = binary_agg_df[binary_agg_df['label'] == 1]
negative_samples = binary_agg_df[binary_agg_df['label'] == 0]

print(f"\n📊 AGGREGATION FEATURES COMPARISON:")
print(f"{'Feature':<40} {'Pos Mean':<12} {'Neg Mean':<12} {'Difference':<12} {'Pos Var':<10} {'Neg Var':<10}")
print("-" * 100)

feature_importance_scores = []

for feature in agg_feature_cols:
    pos_mean = positive_samples[feature].mean()
    neg_mean = negative_samples[feature].mean()
    pos_var = positive_samples[feature].var()
    neg_var = negative_samples[feature].var()
    
    difference = abs(pos_mean - neg_mean)
    
    # Calculate a simple discriminative score
    if pos_var + neg_var > 0:
        discriminative_score = difference / (pos_var + neg_var + 1e-6)
    else:
        discriminative_score = 0
    
    feature_importance_scores.append({
        'feature': feature,
        'pos_mean': pos_mean,
        'neg_mean': neg_mean,
        'difference': difference,
        'discriminative_score': discriminative_score
    })
    
    print(f"{feature:<40} {pos_mean:<12.3f} {neg_mean:<12.3f} {difference:<12.3f} {pos_var:<10.3f} {neg_var:<10.3f}")

# Sort features by discriminative power
feature_importance_df = pd.DataFrame(feature_importance_scores)
feature_importance_df = feature_importance_df.sort_values('discriminative_score', ascending=False)

print(f"\n🎯 TOP 15 MOST DISCRIMINATIVE AGGREGATION FEATURES:")
print(f"{'Feature':<40} {'Discriminative Score':<20} {'Difference':<12}")
print("-" * 75)
for _, row in feature_importance_df.head(15).iterrows():
    print(f"{row['feature']:<40} {row['discriminative_score']:<20.6f} {row['difference']:<12.3f}")

=== AGGREGATION FEATURES ANALYSIS ===

Filtered to binary classification: 39 samples
Label distribution: {0: 39}

📊 AGGREGATION FEATURES COMPARISON:
Feature                                  Pos Mean     Neg Mean     Difference   Pos Var    Neg Var   
----------------------------------------------------------------------------------------------------
max_api_aggregation_score                nan          0.846        nan          nan        0.134     
behavioral_api_agg_count                 nan          0.769        nan          nan        0.182     
fp_api_agg_count                         nan          0.077        nan          nan        0.073     
total_aggregation_count                  nan          0.846        nan          nan        0.134     
behavioral_agg_ratio                     nan          0.769        nan          nan        0.182     
fp_agg_ratio                             nan          0.077        nan          nan        0.073     
has_aggregation                     

In [ ]:
# Cell 5: Test Vendor Correlation for Aggregation Features
def test_aggregation_vendor_correlation(features_df, threshold=0.3):
    """
    Test if aggregation features are vendor-agnostic
    """
    print("=== TESTING AGGREGATION FEATURES VENDOR CORRELATION ===")
    
    # Only analyze positive samples (with vendor info)
    positive_features = features_df[features_df['label'] == 1].copy()
    
    if len(positive_features) == 0:
        print("No positive samples to analyze")
        return None, None, None
    
    # Get aggregation feature columns
    agg_feature_cols = [col for col in features_df.columns if col not in ['script_id', 'label', 'vendor']]
    
    # One-hot encode vendors for correlation analysis
    vendor_dummies = pd.get_dummies(positive_features['vendor'], prefix='vendor')
    
    print(f"Testing {len(agg_feature_cols)} aggregation features against {len(vendor_dummies.columns)} vendors")
    print(f"Correlation threshold: {threshold}")
    
    # Calculate correlations
    feature_vendor_pairs = []
    
    for feature in agg_feature_cols:
        max_correlation = 0
        max_vendor = ""
        correlations = []
        
        for vendor_col in vendor_dummies.columns:
            vendor_name = vendor_col.replace('vendor_', '')
            
            # Calculate correlation
            corr = positive_features[feature].corr(vendor_dummies[vendor_col])
            if not np.isnan(corr):
                correlations.append(abs(corr))
                if abs(corr) > max_correlation:
                    max_correlation = abs(corr)
                    max_vendor = vendor_name
            else:
                correlations.append(0)
        
        feature_vendor_pairs.append({
            'feature': feature,
            'max_correlation': max_correlation,
            'worst_vendor': max_vendor,
            'avg_correlation': np.mean(correlations),
            'is_vendor_agnostic': max_correlation < threshold
        })
    
    # Create results DataFrame
    results_df = pd.DataFrame(feature_vendor_pairs).sort_values('max_correlation', ascending=False)
    
    # Summary statistics
    truly_agnostic = results_df[results_df['is_vendor_agnostic'] == True]
    vendor_specific = results_df[results_df['is_vendor_agnostic'] == False]
    
    print(f"\n=== AGGREGATION FEATURES VENDOR CORRELATION RESULTS ===")
    print(f"Features with max correlation < {threshold}: {len(truly_agnostic)}/{len(results_df)} ({len(truly_agnostic)/len(results_df)*100:.1f}%)")
    print(f"Features with max correlation >= {threshold}: {len(vendor_specific)}/{len(results_df)} ({len(vendor_specific)/len(results_df)*100:.1f}%)")
    
    if len(vendor_specific) > 0:
        print(f"\n⚠️  VENDOR-SPECIFIC AGGREGATION FEATURES (correlation >= {threshold}):")
        for _, row in vendor_specific.iterrows():
            print(f"   {row['feature']}: {row['max_correlation']:.3f} (worst: {row['worst_vendor']})")
    
    if len(truly_agnostic) > 0:
        print(f"\n✅ VENDOR-AGNOSTIC AGGREGATION FEATURES (correlation < {threshold}):")
        for _, row in truly_agnostic.head(15).iterrows():
            print(f"   {row['feature']}: {row['max_correlation']:.3f} (avg: {row['avg_correlation']:.3f})")
    
    return results_df, truly_agnostic, vendor_specific

# Test vendor correlation for aggregation features
correlation_results, agnostic_features, vendor_specific_features = test_aggregation_vendor_correlation(
    binary_agg_df, threshold=0.3
)

In [ ]:
# Cell 6: Select Best Aggregation Features
def select_best_aggregation_features(features_df, target_col='label', 
                                    metadata_cols=['script_id', 'label', 'vendor'],
                                    max_features=15, random_state=42):
    """
    Select best aggregation features using multiple criteria
    """
    print("🔍 SELECTING BEST AGGREGATION FEATURES")
    print("=" * 50)
    
    # Prepare data
    feature_cols = [col for col in features_df.columns if col not in metadata_cols]
    X = features_df[feature_cols].copy()
    y = features_df[target_col].copy()
    
    print(f"Starting with {len(feature_cols)} aggregation features")
    
    # STEP 1: Remove low variance features
    print(f"\n🔧 Step 1: Variance Filter")
    variance_selector = VarianceThreshold(threshold=0.01)
    X_var = variance_selector.fit_transform(X)
    features_after_variance = X.columns[variance_selector.get_support()].tolist()
    
    removed_variance = len(feature_cols) - len(features_after_variance)
    print(f"   Removed {removed_variance} low variance features")
    print(f"   Remaining: {len(features_after_variance)}")
    
    X = X[features_after_variance]
    
    # STEP 2: Statistical significance (F-test)
    print(f"\n🔧 Step 2: Statistical Significance")
    k_best = min(max_features + 5, len(features_after_variance))
    stat_selector = SelectKBest(score_func=f_classif, k=k_best)
    X_stat = stat_selector.fit_transform(X, y)
    features_after_stats = X.columns[stat_selector.get_support()].tolist()
    
    print(f"   Selected top {len(features_after_stats)} by F-test")
    
    X = X[features_after_stats]
    
    # STEP 3: Random Forest Importance
    print(f"\n🔧 Step 3: Random Forest Importance")
    rf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    rf.fit(X, y)
    
    # Get feature importances
    importances = rf.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    # Select top features by importance
    final_features = feature_importance_df.head(max_features)['feature'].tolist()
    
    print(f"   Selected top {len(final_features)} by RF importance")
    print(f"   Final aggregation features: {final_features}")
    
    # STEP 4: Vendor agnostic check
    print(f"\n🔧 Step 4: Vendor Agnostic Validation")
    
    # Check which selected features are vendor agnostic
    if correlation_results is not None:
        agnostic_selected = []
        vendor_specific_selected = []
        
        for feature in final_features:
            feature_corr = correlation_results[correlation_results['feature'] == feature]
            if len(feature_corr) > 0:
                is_agnostic = feature_corr.iloc[0]['is_vendor_agnostic']
                max_corr = feature_corr.iloc[0]['max_correlation']
                if is_agnostic:
                    agnostic_selected.append((feature, max_corr))
                else:
                    vendor_specific_selected.append((feature, max_corr))
        
        print(f"   Vendor-agnostic features: {len(agnostic_selected)}")
        for feature, corr in agnostic_selected:
            print(f"     {feature}: max_corr={corr:.3f}")
        
        print(f"   Vendor-specific features: {len(vendor_specific_selected)}")
        for feature, corr in vendor_specific_selected:
            print(f"     {feature}: max_corr={corr:.3f}")
    
    # STEP 5: Quick validation
    print(f"\n🔧 Step 5: Performance Validation")
    X_original = features_df[feature_cols]
    X_selected = features_df[final_features]
    
    # Compare performance
    rf_validator = RandomForestClassifier(n_estimators=50, random_state=random_state)
    
    score_original = cross_val_score(rf_validator, X_original, y, cv=3, scoring='roc_auc').mean()
    score_selected = cross_val_score(rf_validator, X_selected, y, cv=3, scoring='roc_auc').mean()
    
    print(f"   All aggregation features ({len(feature_cols)}): {score_original:.4f}")
    print(f"   Selected features ({len(final_features)}): {score_selected:.4f}")
    print(f"   Difference: {score_selected - score_original:+.4f}")
    
    if score_selected >= score_original - 0.02:
        print(f"   ✅ Aggregation feature selection successful!")
    else:
        print(f"   ⚠️  Performance dropped significantly")
    
    return final_features, feature_importance_df, score_selected

# Select best aggregation features
selected_agg_features, agg_importance_df, agg_performance = select_best_aggregation_features(
    binary_agg_df,
    target_col='label',
    max_features=12,
    random_state=42
)

print(f"\n🎯 SELECTED AGGREGATION FEATURES FOR MODELING:")
for i, feature in enumerate(selected_agg_features, 1):
    importance = agg_importance_df[agg_importance_df['feature'] == feature]['importance'].iloc[0]
    print(f"{i:2d}. {feature:<40} (importance: {importance:.4f})")

In [ ]:
# Cell 7: Load Original Features and Compare
def load_original_features():
    """
    Load the original static features for comparison
    """
    conn = psycopg2.connect(
        host="localhost",
        port=5434,
        database="vv8_backend",
        user="vv8",
        password="vv8"
    )
    
    query = """
    SELECT 
        script_id,
        fingerprinting_source_apis,
        behavioral_source_apis,
        behavioral_source_api_count,
        fingerprinting_source_api_count,
        behavioral_apis_access_count,
        fingerprinting_api_access_count,
        label,
        vendor
    FROM multicore_static_info_known_companies
    """
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    return df

def create_original_static_features(df):
    """
    Create the same vendor-agnostic features as in the original notebook
    (simplified version for comparison)
    """
    features_list = []
    
    for idx, row in df.iterrows():
        try:
            features = {}
            
            # Safe extraction
            behavioral_access = row['behavioral_apis_access_count'] if row['behavioral_apis_access_count'] is not None else {}
            fp_access = row['fingerprinting_api_access_count'] if row['fingerprinting_api_access_count'] is not None else {}
            behavioral_sources = row['behavioral_source_apis'] if row['behavioral_source_apis'] is not None else []
            fp_sources = row['fingerprinting_source_apis'] if row['fingerprinting_source_apis'] is not None else []
            
            # Basic counts
            total_behavioral = len(behavioral_sources) if behavioral_sources is not None else 0
            total_fp = len(fp_sources) if fp_sources is not None else 0
            total_apis = total_behavioral + total_fp
            
            # Key original features
            if total_apis > 0:
                features['behavioral_focus_ratio'] = total_behavioral / total_apis
                features['fp_focus_ratio'] = total_fp / total_apis
            else:
                features['behavioral_focus_ratio'] = 0
                features['fp_focus_ratio'] = 0
            
            # Interaction diversity
            event_types = set()
            if behavioral_sources is not None:
                for api in behavioral_sources:
                    api_str = str(api)
                    if 'MouseEvent' in api_str:
                        event_types.add('mouse')
                    elif 'KeyboardEvent' in api_str:
                        event_types.add('keyboard')
                    elif 'TouchEvent' in api_str or 'Touch.' in api_str:
                        event_types.add('touch')
                    elif 'PointerEvent' in api_str:
                        event_types.add('pointer')
            
            features['interaction_diversity'] = len(event_types)
            
            # Access intensity
            total_behavioral_accesses = sum(behavioral_access.values()) if behavioral_access else 0
            total_fp_accesses = sum(fp_access.values()) if fp_access else 0
            total_accesses = total_behavioral_accesses + total_fp_accesses
            
            features['collection_intensity'] = total_accesses / max(total_apis, 1)
            
            # Binary capabilities
            features['tracks_mouse'] = int(any('MouseEvent' in str(api) for api in behavioral_sources)) if behavioral_sources else 0
            features['tracks_keyboard'] = int(any('KeyboardEvent' in str(api) for api in behavioral_sources)) if behavioral_sources else 0
            features['uses_canvas_fp'] = int(any('Canvas' in str(api) or 'WebGL' in str(api) for api in fp_sources)) if fp_sources else 0
            
            # Store metadata
            features['script_id'] = int(row['script_id'])
            features['label'] = int(row['label'])
            features['vendor'] = row['vendor'] if pd.notna(row['vendor']) else 'negative'
            
            features_list.append(features)
            
        except Exception as e:
            print(f"Error processing script {row.get('script_id', 'unknown')}: {e}")
            continue
    
    return pd.DataFrame(features_list)

# Load and create original features
print("Loading original static features for comparison...")
original_df = load_original_features()
original_features_df = create_original_static_features(original_df)

# Filter to binary classification
binary_original_df = original_features_df[original_features_df['label'].isin([0, 1])].copy()

original_feature_cols = [col for col in binary_original_df.columns if col not in ['script_id', 'label', 'vendor']]
print(f"Created {len(original_feature_cols)} original static features")
print(f"Original features: {original_feature_cols}")

# Compare performance
print(f"\n🔬 PERFORMANCE COMPARISON:")

# Ensure same samples
common_script_ids = set(binary_agg_df['script_id']).intersection(set(binary_original_df['script_id']))
print(f"Common samples: {len(common_script_ids)}")

agg_subset = binary_agg_df[binary_agg_df['script_id'].isin(common_script_ids)].sort_values('script_id').reset_index(drop=True)
orig_subset = binary_original_df[binary_original_df['script_id'].isin(common_script_ids)].sort_values('script_id').reset_index(drop=True)

# Test individual feature sets
rf_tester = RandomForestClassifier(n_estimators=50, random_state=42)

# Original features only
orig_score = cross_val_score(rf_tester, orig_subset[original_feature_cols], orig_subset['label'], cv=5, scoring='roc_auc').mean()

# Selected aggregation features only
agg_score = cross_val_score(rf_tester, agg_subset[selected_agg_features], agg_subset['label'], cv=5, scoring='roc_auc').mean()

print(f"\nIndividual Performance:")
print(f"Original static features only: {orig_score:.4f}")
print(f"Selected aggregation features only: {agg_score:.4f}")
print(f"Aggregation improvement: {agg_score - orig_score:+.4f}")

In [ ]:
# Cell 8: Combine Best Features and Test
def combine_feature_sets(agg_df, orig_df, selected_agg_features, orig_features):
    """
    Combine aggregation and original features
    """
    # Ensure same samples and order
    common_script_ids = set(agg_df['script_id']).intersection(set(orig_df['script_id']))
    
    agg_subset = agg_df[agg_df['script_id'].isin(common_script_ids)].sort_values('script_id').reset_index(drop=True)
    orig_subset = orig_df[orig_df['script_id'].isin(common_script_ids)].sort_values('script_id').reset_index(drop=True)
    
    # Verify same order
    assert (agg_subset['script_id'] == orig_subset['script_id']).all(), "Script IDs don't match!"
    
    # Combine features
    combined_df = orig_subset[['script_id', 'label', 'vendor']].copy()
    
    # Add original features
    for feature in orig_features:
        combined_df[f'orig_{feature}'] = orig_subset[feature]
    
    # Add selected aggregation features
    for feature in selected_agg_features:
        combined_df[f'agg_{feature}'] = agg_subset[feature]
    
    return combined_df

# Combine feature sets
print("Combining original and aggregation features...")
combined_features_df = combine_feature_sets(
    agg_subset, orig_subset, selected_agg_features, original_feature_cols
)

combined_feature_cols = [col for col in combined_features_df.columns if col not in ['script_id', 'label', 'vendor']]
print(f"Combined features: {len(combined_feature_cols)} total")
print(f"  - Original features: {len(original_feature_cols)}")
print(f"  - Aggregation features: {len(selected_agg_features)}")

# Test combined performance
print(f"\n🚀 TESTING COMBINED FEATURES:")

rf_combined = RandomForestClassifier(n_estimators=100, random_state=42)
combined_score = cross_val_score(rf_combined, combined_features_df[combined_feature_cols], 
                                combined_features_df['label'], cv=5, scoring='roc_auc').mean()

print(f"\nPerformance Comparison:")
print(f"Original static features only:        {orig_score:.4f}")
print(f"Aggregation features only:            {agg_score:.4f}")
print(f"Combined features:                    {combined_score:.4f}")
print(f"\nImprovements:")
print(f"Aggregation vs Original:              {agg_score - orig_score:+.4f}")
print(f"Combined vs Original:                 {combined_score - orig_score:+.4f}")
print(f"Combined vs Aggregation only:         {combined_score - agg_score:+.4f}")

# Feature importance in combined model
rf_combined.fit(combined_features_df[combined_feature_cols], combined_features_df['label'])
combined_importance = pd.DataFrame({
    'feature': combined_feature_cols,
    'importance': rf_combined.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔝 TOP 15 FEATURES IN COMBINED MODEL:")
for idx, row in combined_importance.head(15).iterrows():
    feature_type = "AGG" if row['feature'].startswith('agg_') else "ORIG"
    clean_name = row['feature'].replace('agg_', '').replace('orig_', '')
    print(f"{feature_type:<5} {clean_name:<40} {row['importance']:.4f}")

# Count feature types in top features
top_15 = combined_importance.head(15)
agg_in_top = sum(1 for f in top_15['feature'] if f.startswith('agg_'))
orig_in_top = sum(1 for f in top_15['feature'] if f.startswith('orig_'))

print(f"\nTop 15 feature composition:")
print(f"  Aggregation features: {agg_in_top}/15 ({agg_in_top/15*100:.1f}%)")
print(f"  Original features: {orig_in_top}/15 ({orig_in_top/15*100:.1f}%)")

In [ ]:
# Cell 9: Vendor-Aware Testing with Combined Features
def create_vendor_aware_split(features_df, test_size=0.3, random_state=42):
    """
    Create vendor-aware train/test split
    """
    np.random.seed(random_state)
    
    # Separate positives and negatives
    positives = features_df[features_df['label'] == 1].copy()
    negatives = features_df[features_df['label'] == 0].copy()
    
    print(f"Splitting {len(positives)} positives and {len(negatives)} negatives...")
    
    # Analyze positive vendor distribution
    vendor_counts = positives['vendor'].value_counts()
    high_volume_vendors = vendor_counts[vendor_counts > 20].index.tolist()
    medium_volume_vendors = vendor_counts[(vendor_counts >= 5) & (vendor_counts <= 20)].index.tolist()
    low_volume_vendors = vendor_counts[vendor_counts < 5].index.tolist()
    
    train_pos_indices = []
    test_pos_indices = []
    
    # High volume vendors: Split scripts within vendor
    for vendor in high_volume_vendors:
        vendor_scripts = positives[positives['vendor'] == vendor].index.tolist()
        np.random.shuffle(vendor_scripts)
        
        n_test = max(1, int(len(vendor_scripts) * test_size))
        test_pos_indices.extend(vendor_scripts[:n_test])
        train_pos_indices.extend(vendor_scripts[n_test:])
    
    # Medium volume vendors: Split by vendor
    np.random.shuffle(medium_volume_vendors)
    n_train_vendors = max(1, int(len(medium_volume_vendors) * 0.6))
    
    train_medium_vendors = medium_volume_vendors[:n_train_vendors]
    test_medium_vendors = medium_volume_vendors[n_train_vendors:]
    
    for vendor in train_medium_vendors:
        vendor_scripts = positives[positives['vendor'] == vendor].index.tolist()
        train_pos_indices.extend(vendor_scripts)
    
    for vendor in test_medium_vendors:
        vendor_scripts = positives[positives['vendor'] == vendor].index.tolist()
        test_pos_indices.extend(vendor_scripts)
    
    # Low volume vendors: Split by vendor
    np.random.shuffle(low_volume_vendors)
    n_test_low_vendors = len(low_volume_vendors) // 2
    
    train_low_vendors = low_volume_vendors[n_test_low_vendors:]
    test_low_vendors = low_volume_vendors[:n_test_low_vendors]
    
    for vendor in train_low_vendors:
        vendor_scripts = positives[positives['vendor'] == vendor].index.tolist()
        train_pos_indices.extend(vendor_scripts)
    
    for vendor in test_low_vendors:
        vendor_scripts = positives[positives['vendor'] == vendor].index.tolist()
        test_pos_indices.extend(vendor_scripts)
    
    # Split negatives randomly
    neg_indices = negatives.index.tolist()
    np.random.shuffle(neg_indices)
    n_test_neg = int(len(neg_indices) * test_size)
    
    train_neg_indices = neg_indices[n_test_neg:]
    test_neg_indices = neg_indices[:n_test_neg]
    
    # Combine indices
    train_indices = train_pos_indices + train_neg_indices
    test_indices = test_pos_indices + test_neg_indices
    
    print(f"Final split:")
    print(f"Train: {len(train_pos_indices)} positives + {len(train_neg_indices)} negatives = {len(train_indices)} total")
    print(f"Test: {len(test_pos_indices)} positives + {len(test_neg_indices)} negatives = {len(test_indices)} total")
    
    return train_indices, test_indices

def create_vendor_weights_fixed(features_df, train_idx):
    """Create inverse frequency weights for positive vendors"""
    train_df = features_df.loc[train_idx]
    train_positives = train_df[train_df['label'] == 1]
    
    if len(train_positives) == 0:
        return np.ones(len(train_idx))
    
    vendor_counts = train_positives['vendor'].value_counts()
    vendor_weights = 1 / np.sqrt(vendor_counts)
    vendor_weights = vendor_weights / vendor_weights.sum() * len(vendor_weights)
    
    sample_weights = np.ones(len(train_idx))
    for i, idx in enumerate(train_idx):
        row = features_df.loc[idx]
        if row['label'] == 1 and row['vendor'] in vendor_weights:
            sample_weights[i] = vendor_weights[row['vendor']]
    
    return sample_weights

# Test combined features with vendor-aware splitting
print("=== VENDOR-AWARE TESTING WITH COMBINED FEATURES ===")

# Create vendor-aware split
train_idx, test_idx = create_vendor_aware_split(combined_features_df)

# Prepare data
X_train = combined_features_df.loc[train_idx, combined_feature_cols]
y_train = combined_features_df.loc[train_idx, 'label']
X_test = combined_features_df.loc[test_idx, combined_feature_cols]
y_test = combined_features_df.loc[test_idx, 'label']

# Create sample weights
sample_weights = create_vendor_weights_fixed(combined_features_df, train_idx)

print(f"\nTraining set: {len(train_idx)} samples")
print(f"Test set: {len(test_idx)} samples")

# Train models with different feature sets
models = {
    'Original Features': {
        'features': [col for col in combined_feature_cols if col.startswith('orig_')],
        'model': RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, class_weight='balanced')
    },
    'Aggregation Features': {
        'features': [col for col in combined_feature_cols if col.startswith('agg_')],
        'model': RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, class_weight='balanced')
    },
    'Combined Features': {
        'features': combined_feature_cols,
        'model': RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, class_weight='balanced')
    }
}

results = {}

for model_name, config in models.items():
    print(f"\n--- Testing {model_name} ---")
    
    features = config['features']
    model = config['model']
    
    # Train model
    X_train_subset = X_train[features]
    X_test_subset = X_test[features]
    
    model.fit(X_train_subset, y_train, sample_weight=sample_weights)
    
    # Predictions
    y_pred = model.predict(X_test_subset)
    y_pred_proba = model.predict_proba(X_test_subset)[:, 1]
    
    # Metrics
    accuracy = model.score(X_test_subset, y_test)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    results[model_name] = {
        'accuracy': accuracy,
        'auc': auc,
        'features_count': len(features)
    }
    
    print(f"Features: {len(features)}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"ROC AUC: {auc:.4f}")

# Summary comparison
print(f"\n🏆 FINAL VENDOR-AWARE RESULTS SUMMARY:")
print(f"{'Model':<25} {'Features':<10} {'Accuracy':<10} {'ROC AUC':<10}")
print("-" * 60)

for model_name, metrics in results.items():
    print(f"{model_name:<25} {metrics['features_count']:<10} {metrics['accuracy']:<10.4f} {metrics['auc']:<10.4f}")

# Calculate improvements
orig_auc = results['Original Features']['auc']
agg_auc = results['Aggregation Features']['auc']
combined_auc = results['Combined Features']['auc']

print(f"\n📈 IMPROVEMENTS:")
print(f"Aggregation vs Original: {agg_auc - orig_auc:+.4f} AUC")
print(f"Combined vs Original: {combined_auc - orig_auc:+.4f} AUC")
print(f"Combined vs Aggregation: {combined_auc - agg_auc:+.4f} AUC")

if combined_auc > max(orig_auc, agg_auc):
    print(f"\n✅ Combined features provide the best performance!")
elif agg_auc > orig_auc:
    print(f"\n✅ Aggregation features outperform original features!")
else:
    print(f"\n⚠️  Original features still perform best.")

In [ ]:
# Cell 10: Summary and Recommendations
print("=" * 80)
print("🎯 AGGREGATION FEATURES ANALYSIS SUMMARY")
print("=" * 80)

print(f"\n📊 DATASET OVERVIEW:")
print(f"  Total scripts analyzed: {len(df):,}")
print(f"  Binary classification samples: {len(binary_agg_df):,}")
print(f"  Positive samples (behavioral biometrics): {len(binary_agg_df[binary_agg_df['label']==1]):,}")
print(f"  Negative samples: {len(binary_agg_df[binary_agg_df['label']==0]):,}")

print(f"\n🔧 FEATURE ENGINEERING:")
print(f"  Total aggregation features created: {len(agg_feature_cols)}")
print(f"  Features after variance filtering: {len(selected_agg_features)}")
print(f"  Vendor-agnostic aggregation features: {len(agnostic_features) if agnostic_features is not None else 'N/A'}")
print(f"  Selected aggregation features: {len(selected_agg_features)}")

print(f"\n📈 PERFORMANCE RESULTS:")
if 'results' in locals():
    orig_perf = results['Original Features']
    agg_perf = results['Aggregation Features']
    combined_perf = results['Combined Features']
    
    print(f"  Original static features:")
    print(f"    - Features: {orig_perf['features_count']}")
    print(f"    - Accuracy: {orig_perf['accuracy']:.4f}")
    print(f"    - ROC AUC: {orig_perf['auc']:.4f}")
    
    print(f"  Aggregation features only:")
    print(f"    - Features: {agg_perf['features_count']}")
    print(f"    - Accuracy: {agg_perf['accuracy']:.4f}")
    print(f"    - ROC AUC: {agg_perf['auc']:.4f}")
    print(f"    - Improvement over original: {agg_perf['auc'] - orig_perf['auc']:+.4f} AUC")
    
    print(f"  Combined features:")
    print(f"    - Features: {combined_perf['features_count']}")
    print(f"    - Accuracy: {combined_perf['accuracy']:.4f}")
    print(f"    - ROC AUC: {combined_perf['auc']:.4f}")
    print(f"    - Improvement over original: {combined_perf['auc'] - orig_perf['auc']:+.4f} AUC")

print(f"\n🏆 TOP AGGREGATION FEATURES:")
if 'selected_agg_features' in locals():
    for i, feature in enumerate(selected_agg_features[:10], 1):
        print(f"  {i:2d}. {feature}")

print(f"\n💡 KEY INSIGHTS:")

# Determine if aggregation features are useful
if 'results' in locals():
    agg_improvement = results['Aggregation Features']['auc'] - results['Original Features']['auc']
    combined_improvement = results['Combined Features']['auc'] - results['Original Features']['auc']
    
    if agg_improvement > 0.02:
        print(f"  ✅ Aggregation features provide significant improvement ({agg_improvement:+.4f} AUC)")
    elif agg_improvement > 0.005:
        print(f"  ✅ Aggregation features provide modest improvement ({agg_improvement:+.4f} AUC)")
    else:
        print(f"  ⚠️  Aggregation features provide minimal improvement ({agg_improvement:+.4f} AUC)")
    
    if combined_improvement > agg_improvement:
        print(f"  ✅ Combining aggregation and original features is beneficial")
    else:
        print(f"  ⚠️  Combining features doesn't provide additional benefit")

# Vendor agnosticism
if agnostic_features is not None:
    agnostic_pct = len(agnostic_features) / len(agg_feature_cols) * 100
    if agnostic_pct > 70:
        print(f"  ✅ Most aggregation features are vendor-agnostic ({agnostic_pct:.1f}%)")
    elif agnostic_pct > 50:
        print(f"  ⚠️  Moderate vendor-agnostic aggregation features ({agnostic_pct:.1f}%)")
    else:
        print(f"  ❌ Many aggregation features are vendor-specific ({agnostic_pct:.1f}%)")

print(f"\n🎯 RECOMMENDATIONS:")

if 'results' in locals():
    best_model = max(results.keys(), key=lambda k: results[k]['auc'])
    print(f"  1. Use {best_model.lower()} for best performance")
    
    if results['Aggregation Features']['auc'] > results['Original Features']['auc']:
        print(f"  2. Aggregation features are valuable - include them in your model")
    else:
        print(f"  2. Aggregation features show limited benefit - focus on original features")
    
    if results['Combined Features']['auc'] == max(results[k]['auc'] for k in results):
        print(f"  3. Combine aggregation and original features for optimal performance")

print(f"\n📝 SELECTED AGGREGATION FEATURES FOR INTEGRATION:")
if 'selected_agg_features' in locals():
    for feature in selected_agg_features:
        print(f"  - {feature}")

print(f"\n✅ Analysis complete! Aggregation features {'are' if agg_improvement > 0.01 else 'may be'} beneficial for your behavioral biometrics detection model.")